In [1]:
import os
import sys
print(os.getcwd())

root_dir = os.path.abspath(
    os.path.join(
        os.path.dirname(os.getcwd()), '..'
    )
)
print(root_dir)
sys.path.insert(0, root_dir)

/media/tinhanhnguyen/Data1/Projects/HCMAI/local/playground
/media/tinhanhnguyen/Data1/Projects/HCMAI


In [33]:
from src.processing.video_processing.main import process_single_video, KeyFrameExtractor
from src.processing.video_processing.autoshot import AutoShot

from src.processing.audio_extraction.audio_extraction import extract_audio
from pathlib import Path

In [34]:

pretrained_model_path = "/media/tinhanhnguyen/Data1/Projects/HCMAI/local/transnetv2-pytorch-weights.pth"
video_sample = "/media/tinhanhnguyen/Data1/Projects/HCMAI/data/videos/Videos_L01_a/video/L01_V003.mp4"
shot_detector = AutoShot(pretrained_model_path)
keyFrameExtractor = KeyFrameExtractor('.')

Loading the pretrained model from /media/tinhanhnguyen/Data1/Projects/HCMAI/local/transnetv2-pytorch-weights.pth


In [ ]:
# scenes = process_single_video(video_sample, shot_detector, keyFrameExtractor, return_keyframes=False)

In [36]:
import json

# with open('./scene.json', 'w') as f:
#     json.dump(scenes, f, indent=4)

scenes = json.load(open('./scene.json'))



In [ ]:
import cv2
from tqdm import tqdm
def frame_to_time(fps: float, frame_number: int) -> str:
    total_seconds = frame_number / fps
    print(total_seconds)
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def get_video_fps(video_path: str) -> float:
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video file: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return fps




In [108]:
get_video_fps(video_sample)

25.0

In [38]:
def extract_frame(video_path: str, frame_number: int):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_number >= total_frames:
        raise ValueError(f"Frame number {frame_number} exceeds total frames {total_frames}")

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    ret, frame = cap.read()
    cap.release()

    if not ret:
        raise RuntimeError(f"Failed to read frame at position {frame_number}")

    return frame


import matplotlib.pyplot as plt
 

def scenes_display(video_path: str, scene):
    frame_start, frame_end = scene

    plt.figure(figsize=(15, (frame_end - frame_start)//5 + 3))

    for idx, frame_number in tqdm(enumerate(range(frame_start, frame_end + 1))):
        frame = extract_frame(video_path, frame_number)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib

        plt.subplot((frame_end - frame_start)//5 + 1, 5, idx + 1)  # 5 images per row
        plt.imshow(frame_rgb)
        plt.title(f"Frame {frame_number}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [54]:
from dotenv import load_dotenv
load_dotenv()
from llama_index.core.prompts import PromptTemplate
from llama_index.core.bridge.pydantic import BaseModel, Field

In [58]:
transcript = json.load(open('/media/tinhanhnguyen/Data1/Projects/HCMAI/local/playground/L01_V003.json', 'r'))

In [112]:
from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(
    model="models/gemini-2.5-flash-lite-preview-06-17",
    api_key=os.getenv('GEMINI_API_KEY'),  
)

In [113]:
class EventItem(BaseModel):
    """Thông tin của một sự kiện được trích xuất từ video."""
    eve: str = Field(..., description="Tiêu đề sự kiện")
    sum: str = Field(..., description="Tóm tắt nội dung sự kiện")
    ts: list[str] = Field(..., description="Thời gian bắt đầu và kết thúc dưới dạng frame numbers")
    cnt: str = Field(..., description="Nội dung giọng nói, đã được chỉnh sửa chính tả")


class EventList(BaseModel):
    """Danh sách các sự kiện trong video giảng dạy."""
    events: list[EventItem]

In [115]:


requirements_prompt = PromptTemplate("""


Bạn là một người biên tập nội dung thời sự chuyên nghiệp. Bạn sẽ được cung cấp một đoạn ASR gồm có lời thoại tin thức thời sự và thông tin thời gian. Đây là ví dụ:

00:00:05:520 - 00:00:16:960:  kính chào quý vị rất hân hạnh được gặp lại quý vị trong chương trình sáu mươi giây của đài truyền hình thành phố hồ chí minh chương trình sáng nay thứ ba ngày ba mươi mốt tháng mười sẽ chuyển đến quý vị những nội dung đáng chú ý sau đây

Trong đó: 
1. 00:00:05:520: Thời gian bắt đầu giọng nói
2. 00:00:16:960: Thời gian kết thúc giọng nói
3. Và nội dung giọng nói.

Các chủ đề sẽ liên quan tới tin tức thời sự trong và ngoài nước.



1. Phân tích nội dung và chia thành các sự kiện độc lập
- Vì đây là nội dung thời sự, mỗi sự kiện thường độc lập với nhau.
- Dựa trên nội dung, bạn hãy xác định ranh giới giữa các sự kiện và chia đoạn ASR thành các đoạn văn tương ứng với từng sự kiện riêng biệt.
- Mỗi sự kiện cần có:
        - Thời gian bắt đầu và kết thúc (có thể ước chừng).
        - Nội dung tương ứng.
- Không có sự chồng lấn thời gian giữa các sự kiện. Điểm kết thúc của sự kiện này là điểm bắt đầu của sự kiện tiếp theo.
- Giữa các sự kiện nên có một khoảng thời gian để phân cách rõ ràng

2. Xử lý đoạn giao thoa giữa hai sự kiện:
- Nếu một sự kiện kết thúc giữa một câu hoặc một đoạn, hãy tách nội dung đó ra cho phù hợp với từng sự kiện.
- Thời gian chia tách nên được ước lượng chính xác nhất có thể.

3.Ở mỗi sự kiện, hãy làm cho những câu theo đúng chuẩn tiếng việt hơn(như in hoa tên riêng, dấu chấm dấu phẩy, lọc ra những câu ko liên quan, ...). Tôi muốn bạn giữ lại giọng nói nội dung, giữ tất cả các thông tin(đặc biệt là các con số, thông tin về thời gian cột mốc cái diễn biến ). Và cho tôi một câu tóm tắt cho từng sự kiện đó. 

4. Những đoạn sự kiện không được overlap với nhau. Một số thời gian sẽ giới thiệu nhiều sự kiện cùng lúc, hãy bỏ qua chúng.



Nhiệm vụ: Trích xuất các sự kiện từ transcript sau. Mỗi sự kiện cần bao gồm:
- "eve": Tiêu đề sự kiện
- "sum": Tóm tắt nội dung sự kiện
- "ts": ["frame bắt đầu", "frame kết thúc"]
- "cnt": Nội dung giọng nói đã chỉnh sửa chính tả
Mỗi nội dung sẽ có cấu trúc như sau:

```json
[
    {
        "eve": "<Tiêu đề sự kiện>",
        "sum": "<Tóm tắt nội dung sự kiên>"
        "ts": ["<start frame number>", "<end frame number>"],
        "cnt": "<Nội dung giọng nói, để nguyên, có chỉnh sửa chính tả>",
    }, 
    {
        "eve": "<Tiêu đề sự kiện>",
        "sum": "<Tóm tắt nội dung sự kiên>"
        "ts": ["<start frame number>", "<end frame number>"],
        "cnt": "<Nội dung giọng nói, để nguyên, có chỉnh sửa chính tả>",
    }, 
    {
        "eve": "<Tiêu đề sự kiện>",
        "sum": "<Tóm tắt nội dung sự kiên>"
        "ts": ["<start frame number>", "<end frame number>"],
        "cnt": "<Nội dung giọng nói, để nguyên, có chỉnh sửa chính tả>",
    }, 
]
```

Transcript:
{transcript}

Chỉ trả về JSON hợp lệ theo đúng schema.
""")



In [116]:
result = llm.structured_predict(
    EventList,
    requirements_prompt,
    transcript=transcript
)

In [117]:
result_json = result.model_dump()

In [119]:

# with open('chunked_events.json', 'w', encoding='utf-8') as f:
#     json.dump(result_json, f, indent=4, ensure_ascii=False)

result_json = json.load(open('./chunked_events.json'))
result_json.keys()

dict_keys(['events'])

In [120]:
for res in result_json['events']:
    start_ts = int(res['ts'][0]) 
    end_ts = int(res['ts'][1]) 

    inrange_scenes = list(filter(
        lambda x:  start_ts <= int(x[0]) <= end_ts ,
        scenes
    ))
    res['related_scenes'] = inrange_scenes

In [121]:
result_json['events'][0]

{'eve': 'Trao giải cuộc thi sinh viên với an toàn thông tin ASIAN 2023',
 'sum': 'Lễ tổng kết và trao giải vòng chung khảo cuộc thi sinh viên với an toàn thông tin ASIAN 2023 đã diễn ra tại Hà Nội. Đội tuyển Học viện Kỹ thuật Mật mã cơ sở Hà Nội đã giành giải nhất.',
 'ts': ['00000862', '00002430'],
 'cnt': 'Kính thưa quý vị, tối qua tại Hà Nội đã diễn ra lễ tổng kết và trao giải vòng chung khảo cuộc thi sinh viên với an toàn thông tin ASIAN năm 2023. Cuộc thi do Hiệp hội An toàn thông tin Việt Nam phối hợp với Bộ Giáo dục và Đào tạo, Bộ Thông tin và Truyền thông tổ chức. Vượt qua tám mươi đội thi Việt Nam và ba mươi lăm đội đến từ chín nước ASEAN, đội tuyển của Học viện Kỹ thuật Mật mã cơ sở Hà Nội đã xuất sắc giành giải nhất bài thi dạng CTF và đại diện Việt Nam tham dự cuộc thi Cyber Security Challenge 2023 sắp tới. Ba giải nhì đã được trao cho ba đội đến từ Học viện Kỹ thuật Mật mã cơ sở Thành phố Hồ Chí Minh, Đại học Duy Tân và Học viện Kỹ thuật Quân sự. Đây là lần thứ mười sáu cu

In [123]:
folder_sample = './sample'
cap = cv2.VideoCapture(video_sample)



for idx, event in enumerate(result_json['events']):
    folder_event_name = f'event_{idx}'
    folder_event_path = os.path.join(folder_sample, folder_event_name)

    Path(folder_event_path).mkdir(parents=True, exist_ok=True)
    print("event--")
    for scene in event['related_scenes']:
        start_frame, end_frame = scene
        middle_frame = (start_frame + end_frame) // 2
        print(start_frame,end_frame, middle_frame)

        cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame)

        ret, frame = cap.read()
        
        output_filename = f'scene_{middle_frame}.jpg'
        output_path = os.path.join(folder_event_path, output_filename)

        cv2.imwrite(output_path, frame)
    break
cap.release()


event--
1274 1321 1297
1322 1419 1370
1420 1514 1467
1515 1578 1546
1579 1673 1626
1674 1745 1709
1746 1943 1844
1944 2029 1986
2030 2107 2068
2108 2181 2144
2182 2273 2227
2274 2319 2296
2320 2337 2328
2338 2374 2356
2375 2413 2394
2414 2524 2469


# PROMPT

In [ ]:
detail_input_prompt = """
Đầu vào của nhiệm vụ này gồm có:
# - "eve": Tiêu đề sự kiện
# - "sum": Tóm tắt nội dung sự kiện
# - "ts": ["frame bắt đầu", "frame kết thúc"]
# - "cnt": Nội dung về sự kiện đó, bao gồm các thành phần như diễn biến, đoạn hội thoại nhân vật, ...
# - "keyframes": Một dãy các keyframe liên tục theo thời gian chứa thông tin hình ảnh liên quan với sự kiên
"""


detailed_caption_prompt = """
# Mục tiêu:
Tạo ra một caption mô tả đầy đủ và có cấu trúc cho mỗi keyframe, bao gồm tất cả thực thể, quan hệ không gian, hành động và thuộc tính chi tiết để phục vụ cho việc dense caption nhằm cho mục đích tạo ra scene graph

# Cấu trúc caption:
## Bối cảnh và môi trường không gian:
- Loại không gian: (studio truyền hình/sân khấu/phòng họp/ngoài trời/...)
- Điều kiện ánh sáng: (đèn studio/ánh sáng tự nhiên/đèn sân khấu/...)
- Màu sắc môi trường: (màu nền chủ đạo, hiệu ứng ánh sáng)
- Bối cảnh sự kiện: (lễ trao giải/cuộc thi/hội thảo/...)
- Khung hình nền: Màu gì, mang biểu tượng gì, ...

## Thông tin thực thể chi tiết  (Detailed Entity Information)
### Thực thể con người
Nếu như liên quan tới con người, cần có một số thông tin cụ thể như sau:
1. Vị trí người đó đứng trong khung hình (góc trái/phải, trung tâm, phía trước/sau)
2. Đặc điểm ngoại hình(tuổi ước tính, giới tính, kiểu tóc, ...)
3. Trang Phục: : (màu sắc, kiểu dáng cụ thể, phụ kiện)
4. Tư thế: (đứng/ngồi, hướng mặt, vị trí tay chân)
5. Biểu cảm: (vui vẻ/nghiêm túc/tập trung/...)
6. Thuộc tính khác: (đeo kính, thẻ tên, phù hiệu...)
7. Công việc cụ thể ( bạn có thể dựa vào hình ảnh người đó hoặc ASR để dự đoán nghề nghiệp)

### Thực thể không phải con người
Bạn hãy mô tả chi tiết một thực thể không phải con người, tập trung vào các đặc điểm nổi bật và chức năng của nó. Mục tiêu là cung cấp một cái nhìn toàn diện về vật thể như thể nó đang hiện hữu.

#### Yêu cầu mô tả:
##### Đặc điểm vật lý:
1. Hình dạng: Mô tả hình dạng tổng thể (ví dụ: hình hộp chữ nhật, hình trụ, không đều).
2. Kích thước: Ước tính kích thước (ví dụ: nhỏ gọn, lớn, hoặc kích thước tương đối so với vật thể thông thường).
3. Màu sắc: Nêu màu sắc chính và các chi tiết màu sắc phụ.
4. Chất liệu & Kết cấu: Xác định chất liệu cấu tạo (ví dụ: kim loại, gỗ, nhựa, vải) và cảm giác khi chạm vào bề mặt (ví dụ: nhẵn, thô ráp, cứng, mềm).
##### Chức năng & Trạng thái hoạt động:
1. Mục đích chính: Trình bày chức năng hoặc mục đích sử dụng cơ bản của vật thể (ví dụ: để ngồi, để chứa, để hiển thị thông tin, để thu âm).
2. Trạng thái hiện tại: Nếu vật thể có chức năng động, mô tả rõ trạng thái nó đang thực hiện (ví dụ: đang chứa sách, đang hiển thị hình ảnh, đang thu âm giọng nói).
##### Tương tác giác quan (chọn lọc):
- Chỉ đề cập đến các giác quan nghe hoặc ngửi nếu có đặc điểm nổi bật, đặc trưng và có ý nghĩa đối với vật thể hoặc chức năng của nó (ví dụ: âm thanh lách cách khi di chuyển, mùi gỗ mới).
1. Sách:
* Mô tả: Một quyển sách bìa cứng hình chữ nhật, kích thước cỡ A5, bìa màu xanh đậm với tên sách in màu bạc. Bề mặt bìa nhẵn bóng, các trang giấy bên trong mịn và mỏng.
* Chức năng & Trạng thái: Sách được dùng để đọc và chứa đựng thông tin. Nó đang được mở ở trang 50, hiển thị văn bản và hình ảnh minh họa. Khi lật trang, có tiếng sột soạt nhẹ.

2. Bàn phím máy tính:
* Mô tả: Một chiếc bàn phím cơ hình chữ nhật dẹt, màu đen, kích thước khoảng 35cm x 12cm. Các phím bấm làm bằng nhựa có bề mặt hơi nhám, tạo cảm giác chắc chắn khi gõ.
* Chức năng & Trạng thái: Bàn phím được dùng để nhập liệu vào máy tính. Nó đang kết nối và sẵn sàng nhận lệnh. Khi gõ, có tiếng lách cách đặc trưng của phím cơ.

## QUAN HỆ VÀ HÀNH ĐỘNG (Relations & Actions)
Hành động chính: Ai đang làm gì (trao/nhận giải, bắt tay, chụp ảnh...)
Tương tác: Mối quan hệ giữa các người (ai tương tác với ai)
Luồng sự kiện: Trình tự hành động đang diễn ra
Trạng thái cảm xúc: Không khí chung của khung hình

##  THÔNG TIN NGÔN NGỮ VÀ BIỂU TƯỢNG (Language & Symbolic Information) hay còn gọi là text OCR
[TEXT_VÀ_BIỂU_TƯỢNG]:

Bạn hãy scan các OCR, và chỉ giữ lại các đoạn OCR nào mà liên quan tới sự kiện đó. 

Text có thể đọc: Ghi chính xác nội dung text trên bảng, logo, chữ viết, và thường liên quan tới các vật thể
Logo/Biểu tượng: Mô tả các logo, biểu tượng có thể nhận diện
Màu sắc ý nghĩa: Màu sắc có tính biểu tượng (đỏ-trắng-xanh cho quốc gia...)



## Ngôn ngữ:
Sử dụng tiếng Việt chuẩn
Thuật ngữ nhất quán
Tránh từ mơ hồ ("một số", "có vẻ", "dường như")
Ưu tiên độ chính xác và chi tiết


# Yêu cầu về hình thức văn bản:

- Kết quả đầu ra **phải là đoạn văn miêu tả tự nhiên, giống như cách một nhà báo hoặc người dẫn chương trình miêu tả cảnh quay**.
- **Không sử dụng bullet points, không dùng dấu "-" hay liệt kê đầu dòng**.
- **Tích hợp tất cả thông tin vào trong các câu văn đầy đủ, có liên kết tự nhiên giữa các câu**.
- Đảm bảo giọng văn mạch lạc, giàu thông tin nhưng vẫn tự nhiên như mô tả hiện trường hoặc bình luận trực tiếp.
- Giữ nguyên độ chi tiết và đầy đủ thông tin theo checklist, nhưng phải thể hiện qua các đoạn văn liền mạch.
Ví dụ: 
 1. Một cảnh quay toàn cảnh cho thấy một khu vực rộng lớn bị ngập lụt nghiêm trọng dưới bầu trời u ám. Một vài người đang đứng trong làn nước đục ngầu, một chiếc xe tải đã bị ngập một phần ở phía xa. Phía trên cùng bên phải là logo "HTV9 HD" và thời gian "06:31:09". Dòng chữ bên dưới cho thấy có những người bị lũ cuốn trôi, ám chỉ về một thảm họa thiên nhiên.
2. Một cảnh quan thiên nhiên với dòng sông nước chảy xiết và đục ngầu, có lẽ là do mưa lớn gây lũ. Một đám cây xanh tươi mọc lên từ bờ sông
"""



detailed_scene_graph_prompt = """
Nhiệm vụ: Bạn phải tạo ra một scene graph hoàn chỉnh và chất lượng, chi tiết dựa trên visual caption mà bạn đã generate ra. 

# Nguyên tắc xây dựng Scene Graph chất lượng:
1. Cấu trúc phân cấp và mối quan hệ rõ ràng: Scene graph nên phản ánh rõ ràng mối quan hệ giữa các đối tượng (ví dụ: "người A đang đứng cạnh người B", "micro đang được cầm bởi người A", "logo nằm ở góc trên bên trái"). Cấu trúc này giúp mô hình hiểu được bố cục không gian và tương tác.
2. Độ chi tiết cao: Mỗi thực thể (entity) cần được mô tả đầy đủ các thuộc tính quan trọng (màu sắc, hình dạng, chất liệu, hành động, tư thế, biểu cảm, vai trò, v.v.).
3. Nhất quán trong cách đặt tên và mô tả: Sử dụng các thuật ngữ nhất quán cho các loại đối tượng, thuộc tính và mối quan hệ. Điều này rất quan trọng cho việc tổng quát hóa và truy vấn dữ liệu sau này.
4. Bao quát toàn bộ khung cảnh: Cố gắng bao gồm tất cả các đối tượng và yếu tố quan trọng trong khung hình, bao gồm cả bối cảnh và các yếu tố văn bản/logo.
5. Phản ánh hành động và trạng thái: Mô tả không chỉ có gì trong cảnh mà còn ai đang làm gì, đang ở trạng thái nào.
6. Bổ sung Tên riêng và Định danh (Identity Enrichment):Nguyên tắc: Khi ASR cung cấp tên riêng của một người, một địa điểm, hoặc một nhãn cụ thể cho một vật thể, hãy thêm thông tin này vào thuộc tính identity hoặc một trường mới dành riêng cho định danh của thực thể.
Mục đích: Giúp phân biệt rõ ràng các thực thể có cùng loại nhưng khác nhau về danh tính. Ví dụ, phân biệt "một ca sĩ" với "ca sĩ Mỹ Tâm".
7.Làm rõ Bản chất Vật thể từ ASR (Object Clarification from ASR):Nguyên tắc: Nếu ASR cung cấp thông tin về loại vật thể hoặc chức năng đặc biệt của nó mà không rõ ràng chỉ từ hình ảnh, hãy sử dụng thông tin ASR để bổ sung hoặc thay thế các mô tả vật lý.
Mục đích: Cung cấp thông tin sâu hơn về vật thể. Ví dụ, nếu hình ảnh chỉ có các khối màu nhưng ASR nói đó là "pháo hoa", thông tin này rất quan trọng để hiểu ngữ cảnh.

8. Liên kết giữa Âm thanh và Hình ảnh (Audio-Visual Linking):Nguyên tắc: Duy trì mối liên kết rõ ràng giữa các đoạn ASR và các thực thể mà chúng mô tả. Điều này có thể thực hiện bằng cách sử dụng các ID tương ứng hoặc gán nhãn thời gian.
Mục đích: Cho phép truy vấn ngược lại từ ASR đến các thực thể hình ảnh hoặc ngược lại, giúp xác minh và làm giàu dữ liệu.

9. Sử dụng các thông tin ngữ cảnh nhằm tăng tính đúng đắn: iên kết không chỉ dựa trên sự khớp từ khóa trực tiếp mà còn dựa vào ngữ cảnh rộng hơn của cuộc nói chuyện và cảnh quay.
Độ gần về thời gian (Temporal Proximity): Các đoạn ASR thường liên quan đến các hành động hoặc thực thể đang hiển thị gần thời điểm đó trong video.
Độ gần về không gian (Spatial Proximity): Nếu một thực thể được nhắc đến và nó xuất hiện gần một thực thể khác đang hoạt động (ví dụ: "Ngọc Anh đang hát" và Ngọc Anh đang cầm micro), mối liên kết này trở nên mạnh mẽ hơn.
Luồng hội thoại (Discourse Coherence): Theo dõi luồng của cuộc trò chuyện hoặc bài phát biểu để hiểu mối liên hệ giữa các câu.

Một khung sườn ví dụ cụ thể

```json
{
  "frameNumber": "<Số thứ tự của khung hình (ví dụ: 100)>",
  "sceneId": "<UUID duy nhất cho scene này (ví dụ: 'sg_frame_100_abc123')>",
  "sceneDescription": {
    "type": "<Loại không gian (ví dụ: 'sân khấu', 'phòng họp', 'ngoài trời')>",
    "environmentDetails": {
      "lighting": "<Chi tiết về ánh sáng (ví dụ: 'đèn sân khấu', 'ánh sáng tự nhiên', 'đèn trần')>",
      "timeOfDay": "<Thời điểm trong ngày (ví dụ: 'sáng', 'tối', 'trưa')>",
      "colorPalette": ["<màu chủ đạo 1 (ví dụ: 'xanh dương')>", "<màu chủ đạo 2 (ví dụ: 'đỏ')>"],
      "eventContext": "<Ngữ cảnh sự kiện (ví dụ: 'buổi biểu diễn', 'tin tức', 'hội thảo')>",
      "backgroundElements": [
        {
          "description": "<Mô tả chi tiết phông nền (ví dụ: 'phông nền sặc sỡ với họa tiết dân gian')>",
          "colors": ["<màu 1 (ví dụ: 'xanh dương')>", "<màu 2 (ví dụ: 'đỏ')>"],
          "symbolism": "<Ý nghĩa biểu tượng (ví dụ: 'văn hóa truyền thống Việt Nam')>"
        }
      ]
    },
    "contextMood": "<Không khí chung (ví dụ: 'vui vẻ', 'nghiêm túc', 'trang trọng')>"
  },
  "entities": [
    {
      "entityId": "<UUID duy nhất cho thực thể (ví dụ: 'ent_person_abc1')>",
      "type": "<Loại thực thể chính (ví dụ: 'person', 'object', 'apparel', 'text_overlay', 'logo')>",
      "subtype": "<Loại chi tiết hơn (ví dụ: 'singer', 'microphone', 'ao_dai', 'program_info')>",
      "attributes": {
        "identity": {
          "name": "<Tên riêng được xác định (ví dụ: 'Ngọc Anh')>",
          "label": "<Nhãn chung (ví dụ: 'ca sĩ', 'áo dài', 'kính')>",
          "brand": "<Thương hiệu nếu có (ví dụ: 'Roland')>"
        },
        "position": {
          "2Dposition": "<Vị trí tương đối 2D (ví dụ: 'trung tâm', 'góc trên bên trái', 'phía trước')>",
          "3Dposition": "<Vị trí tương đối 3D so với scene (ví dụ: 'tiền cảnh', 'hậu cảnh', 'trung cảnh')>"
        },
        "visual": { // Hợp nhất thuộc tính hình ảnh cho mọi thực thể (người, vật, quần áo, v.v.)
          "color": ["<màu 1 (ví dụ: 'tím sen')>", "<màu 2 (ví dụ: 'vàng')>"],
          "shape": "<Hình dạng tổng thể (ví dụ: 'humanoid', 'rectangular', 'irregular')>",
          "material": "<Chất liệu (ví dụ: 'vải', 'kim loại', 'nhựa')>",
          "texture": "<Kết cấu bề mặt (ví dụ: 'mịn', 'bóng', 'thô ráp')>",
          "sizeEstimation": "<Ước lượng kích thước (ví dụ: 'small', 'medium', 'large', 'tương đối với người')>",
          "details": "<Mô tả thêm về ngoại hình (ví dụ: 'có hàng cúc', 'biểu tượng số 60')>"
        },
        "humanSpecific": { // Các thuộc tính chỉ áp dụng cho type: "person"
          "gender": "<Giới tính (ví dụ: 'nam', 'nữ')>",
          "ageEstimation": "<Độ tuổi ước tính (ví dụ: '20-25 tuổi', 'trung niên')>",
          "hairStyle": "<Kiểu tóc (ví dụ: 'tóc đen dài', 'tóc hoa râm')>",
          "skinColor": "<Màu da (ví dụ: 'da người')>",
          "posture": "<Tư thế (ví dụ: 'đứng', 'ngồi', 'cầm mic')>",
          "action": "<Hành động đang thực hiện (ví dụ: 'hát', 'chơi đàn', 'vỗ tay')>",
          "facialExpression": "<Biểu cảm (ví dụ: 'vui vẻ', 'tươi cười', 'tập trung')>",
          "role": "<Vai trò (ví dụ: 'ca sĩ', 'nhạc công', 'khán giả')>"
        },
        "functionality": "<Chức năng cơ bản (ví dụ: 'khuếch đại âm thanh', 'hiển thị thông tin', 'nhập liệu')>", // Áp dụng cho object
        "textContent": "<Nội dung OCR nếu có (ví dụ: 'HTV9 HD', '60 Giây')>", // Áp dụng cho text_overlay, logo
        "asrRelatedInfo": { // Thông tin bổ sung từ ASR, nếu có
          "asrSegmentId": "<ID của đoạn ASR liên quan (ví dụ: 'asr_seg_001')>",
          "asrTimestampRange": ["<thời gian bắt đầu (giây)>", "<thời gian kết thúc (giây)>"],
          "asrConfidenceScore": "<Điểm tin cậy của ASR (ví dụ: 0.95)>"
        }
      },
      "provenance": {
        "source": ["<nguồn 1 (ví dụ: 'visual_recognition')>", "<nguồn 2 (ví dụ: 'asr')>"],
        "confidenceScore": "<Điểm tin cậy tổng thể của thực thể (ví dụ: 0.98)>"
      }
    }
  ],
  "relations": [
    {
      "relationId": "<UUID duy nhất cho mối quan hệ (ví dụ: 'rel_wears_person_apparel_1')>",
      "subjectEntityId": "<ID của thực thể chủ thể (ví dụ: 'ent_person_abc1')>",
      "predicateType": "<Loại quan hệ (ví dụ: 'wears', 'holds', 'is_next_to', 'is_part_of', 'displays_text')>",
      "objectEntityId": "<ID của thực thể đối tượng (ví dụ: 'ent_apparel_xyz2')>",
      "relationAttributes": {
        "details": "<Mô tả chi tiết hơn về mối quan hệ (ví dụ: 'bằng tay phải', 'ở phía sau')>",
        "location": "<Vị trí cụ thể của mối quan hệ (ví dụ: 'trên sân khấu')>"
      },
      "provenance": {
        "source": ["<nguồn 1 (ví dụ: 'visual_recognition')>", "<nguồn 2 (ví dụ: 'inferred')>"],
        "confidenceScore": "<Điểm tin cậy của mối quan hệ (ví dụ: 0.9)>"
      }
    }
  ]
}
```
"""


event_caption_prompt = """
Bạn là một hệ thống hiểu video. Nhiệm vụ của bạn là tạo ra một mô tả sự kiện (event caption) thật chi tiết, đầy đủ và tự nhiên dựa trên các thông tin sau:
1. Visual Caption: Là các mô tả về những gì xuất hiện trong từng khung hình (keyframe). Các caption này có thể chứa thông tin về cảnh vật, hành động, đối tượng, trạng thái, v.v.
2. ASR (Automatic Speech Recognition): Là đoạn lời thoại, âm thanh, hoặc hội thoại được trích xuất từ video.

# Yêu cầu: 
- Hãy tổng hợp các visual caption của nhiều khung hình theo trình tự thời gian và kết hợp với thông tin từ ASR để tái hiện lại sự kiện diễn ra một cách mạch lạc, tự nhiên.
- Chú ý đến sự thay đổi về hành động, bối cảnh, trạng thái giữa các khung hình.
- Diễn đạt bằng tiếng Việt rõ ràng, có đầu, có cuối, giống như một đoạn mô tả chi tiết sự kiện trong video (giống người kể chuyện).
- Giữ nguyên các thông tin chi tiết về không gian, đối tượng, hành động, và nội dung hội thoại nếu có.
- Nếu các visual caption và ASR có liên quan, hãy kết nối các thông tin đó lại để mô tả sự kiện một cách đầy đủ nhất.
- Nếu phát hiện các hành động liên tiếp hoặc diễn biến theo thời gian, hãy trình bày theo trình tự logic.
"""

event_scene_graph_prompt = """
Nguyên tắc Xây dựng Scene Graph chất lượng, chi tiết và tổng quát hóa:
Mục tiêu Chính: Mô tả đầy đủ và có cấu trúc các thực thể, quan hệ không gian, hành động và thuộc tính chi tiết trong một khung hình hoặc một đoạn video ngắn/scene, sẵn sàng cho việc liên kết với các scene  Graph tạo thành 1 Knowledge Graph.

2. Chuẩn hóa thông tin đầu vào
Tất cả thực thể (entity): Gán UUID duy nhất, dù là người, vật thể hay tổ chức.

Phân loại actor/entity rõ ràng:

actor_type: person, organization, natural_phenomenon, vehicle, infrastructure

role trong event phải rõ chức năng: gây_ra, thực_hiện, chịu_ảnh_hưởng, can_thiệp

Chuẩn hóa thời gian:

Dùng chuẩn ISO-8601 nếu có thời gian chính xác

Nếu thời gian mơ hồ, cần có text mô tả và ghi chú sự không chính xác

3. Quy tắc mô hình hóa mối quan hệ
Loại quan hệ	Mô tả
Causal Relations (Event Graph)	Sự kiện A gây ra sự kiện B (gây_ra, dẫn_đến)
Spatial Relations (Scene Graph)	Vật thể X ở cạnh, bao quanh, bên trong vật thể Y
Temporal Relations (Event Graph)	Sự kiện A xảy ra trước/sau/cùng lúc với sự kiện B
Actor - Event Relations	Ai là người/thực thể gây ra, thực hiện, hay chịu ảnh hưởng
Entity State Change	Thực thể nào có sự thay đổi trạng thái (bị ngập, bị cô lập, vỡ, mất tích,...)

4. Ràng buộc logic khi xây dựng
Mỗi Event phải có ít nhất:
Tác nhân (actor) hoặc hiện tượng tự nhiên 
Đối tượng bị ảnh hưởng (affected_entities)
Thời gian & địa điểm xác định hoặc ước lưng
Sự kiện có thể gồm nhiều cảnh (scene):
Mỗi scene graph là snapshot không gian-thời gian cụ thể, liên kết với event_id

Cấu trúc:
{
  "eventGraphId": "<UUID duy nhất cho biểu đồ sự kiện này (ví dụ: 'eg_flood_hatinh_2023')>",
  "eventGraphSummary": "<Tóm tắt ngắn gọn về chuỗi sự kiện chính (ví dụ: 'Tình hình lũ lụt nghiêm trọng tại Hà Tĩnh, gây thương vong về người, thiệt hại tài sản và gián đoạn giao thông, đồng thời có các hoạt động cứu hộ và chỉ đạo phòng chống thiên tai.')>",
  "events": [
    {
      "eventId": "<UUID duy nhất cho sự kiện này (ví dụ: 'ev_001_woman_swept_away')>",
      "eventType": "<Loại sự kiện chung (ví dụ: 'thiên_tai', 'cứu_hộ', 'giao_thông_gián_đoạn')>",
      "subtype": "<Loại sự kiện chi tiết hơn (ví dụ: 'bị_lũ_cuốn_trôi', 'sạt_lở_đất', 'cấm_tàu')>",
      "description": "<Mô tả ngắn gọn về sự kiện này (ví dụ: 'Hai phụ nữ bị lũ lụt cuốn trôi khi về nhà')>",
      "timespan": {
        "startTime": "<Thời gian bắt đầu (ISO 8601 hoặc ngôn ngữ tự nhiên, ví dụ: '2023-10-30T10:00:00Z' hoặc 'khoảng 10h ngày 30/10')>",
        "endTime": "<Thời gian kết thúc (ISO 8601 hoặc ngôn ngữ tự nhiên, ví dụ: '2023-11-02T23:59:59Z' hoặc 'đến ngày 2/11')>",
        "duration": "<Thời lượng nếu biết (ví dụ: 'kịp thời', 'đến đêm nay')>"
      },
      "location": {
        "specificName": "<Tên địa điểm cụ thể (ví dụ: 'xã Hà Linh', 'huyện Hương Khê')>",
        "regionProvince": "<Tỉnh/Vùng (ví dụ: 'Hà Tĩnh')>",
        "description": "<Mô tả tương đối về địa điểm nếu tên cụ thể không có (ví dụ: 'các điểm sạt lở trên tuyến đường sắt Bắc - Nam')>"
      },
      "actors": [
        {
          "actorId": "<UUID của tác nhân (ví dụ: 'act_001_flood_water')>",
          "actorType": "<Loại tác nhân (ví dụ: 'person', 'organization', 'natural_phenomenon')>",
          "role": "<Vai trò trong sự kiện này (ví dụ: 'gây_ra', 'thực_hiện', 'chỉ_đạo')>",
          "identity": {
            "name": "<Tên riêng (ví dụ: 'lũ lụt', 'Chính quyền địa phương', 'Trần Anh Tuấn')>",
            "description": "<Mô tả thêm (ví dụ: 'cậu bé 13 tuổi', 'Phó Tổng giám đốc Tổng công ty Đường sắt Việt Nam')>"
          }
        }
      ],
      "affectedEntities": [
        {
          "entityId": "<UUID của thực thể bị ảnh hưởng (ví dụ: 'aff_001_two_women')>",
          "entityType": "<Loại thực thể bị ảnh hưởng (ví dụ: 'person', 'infrastructure', 'property', 'vehicle', 'group')>",
          "statusChange": "<Sự thay đổi trạng thái (ví dụ: 'cuốn_trôi', 'tử_vong', 'bị_cô_lập', 'bị_ngập', 'gián_đoạn')>",
          "identity": {
            "name": "<Tên riêng (ví dụ: 'hai phụ nữ', 'đường Hồ Chí Minh', 'tàu SE1')>",
            "description": "<Mô tả thêm (ví dụ: 'bị ngập sâu từ 30 đến 70cm')>"
          },
          "quantity": "<Số lượng nếu áp dụng (ví dụ: 2, 9, 500)>"
        }
      ],
      "causalRelations": [
        {
          "causeEventId": "<UUID của sự kiện nguyên nhân (ví dụ: 'ev_004_heavy_rain_damage')>",
          "effectEventId": "<UUID của sự kiện hậu quả (ví dụ: 'ev_005_rail_interruption')>",
          "relationType": "<Loại quan hệ nhân quả (ví dụ: 'gây_ra', 'dẫn_đến', 'là_nguyên_nhân_của')>"
        }
      ],
      "relatedEvents": [
        {
          "relatedEventId": "<UUID của sự kiện liên quan (ví dụ: 'ev_005_rail_interruption')>",
          "relationType": "<Loại quan hệ liên quan (ví dụ: 'diễn_ra_cùng_lúc_với', 'khắc_phục_cho')>",
          "description": "<Mô tả thêm về mối liên hệ (ví dụ: 'nhằm khắc phục sự cố gián đoạn đường sắt')>"
        }
      ],
      "outcome": "<Kết quả/Hậu quả cuối cùng của sự kiện (ví dụ: 'thiệt hại nặng nề', 'giải cứu thành công', 'tiếp tục mưa')>"
    }
  ],
  "eventRelations": [
    {
      "event1Id": "<UUID của sự kiện 1 (ví dụ: 'ev_004_heavy_rain_damage')>",
      "event2Id": "<UUID của sự kiện 2 (ví dụ: 'ev_001_woman_swept_away')>",
      "relationType": "<Loại quan hệ giữa các sự kiện (ví dụ: 'caused', 'resulted_in', 'sequentially_followed_by')>"
    }
  ]
}
"""



knowledge_graph  = """
Từ các visual caption, scene graph của từng keyframe, event caption và event graph của toàn bộ sự kiện, bạn hãy tạo một KG tổng quát và mạnh mẽ:
1. Định danh duy nhất cho mọi thực thể và mối quan hệ trên toàn bộ KG.
2. Hợp nhất các thực thể nếu chúng cùng đại diện cho một đối tượng/khái niệm trong thế giới thực, dù được nhắc đến trong hình ảnh, âm thanh hay văn bản.
3. Tích hợp các loại quan hệ từ Scene Graph (spatial, attribute-based) và Event Graph (temporal, causal, participant-based).


# Nguyên tắc Xây dựng Knowledge Graph Cuối cùng:
1. Hợp nhất Thực thể (Entity Resolution/Linking):
- Nguyên tắc cốt lõi: Nếu entity_id từ một Scene Graph (khung hình A) và một entity_id từ Scene Graph (khung hình B) hoặc một actor_id/entity_id từ Event Graph đều đề cập đến cùng một thực thể trong thế giới thực (ví dụ: cùng là "ca sĩ Ngọc Anh"), chúng cần được hợp nhất thành một kg_entity_id duy nhất trong global_entities.
- Thực hiện: Đây là phần khó nhất, yêu cầu thuật toán phức tạp hoặc can thiệp thủ công:
    - Sử dụng tên riêng (name) làm chỉ dấu mạnh nhất.
    - Sử dụng thuộc tính tương đồng (vị trí, màu sắc, trang phục, vai trò) qua các khung hình.
    - Sử dụng ngữ cảnh thời gian và không gian.

2. Chuyển đổi Quan hệ (Relation Conversion):
- Các mối quan hệ từ Scene Graph (ví dụ: wears, holds, is_on) và Event Graph (ví dụ: causes, affected_by, performs) sẽ được chuyển đổi thành các predicate_type trong global_relations.
- Phân loại Predicate: Cố gắng chuẩn hóa các loại quan hệ để chúng nhất quán trên toàn bộ KG.
3. Làm giàu Thuộc tính (Attribute Enrichment):
- Các thuộc tính của global_entities sẽ là tập hợp hoặc tổng hợp của các thuộc tính từ các nguồn khác nhau. Ví dụ, physical_description sẽ đến từ Scene Graph, trong khi role_descriptions có thể đến từ Event Graph hoặc ASR.
4. Theo dõi Nguồn gốc (Provenance Tracking):
- Mỗi kg_entity_id và kg_relation_id cần có một trường source_references hoặc provenance để biết chúng được tạo ra từ đâu (Scene Graph, Event Graph, ASR, OCR, hoặc suy luận). Điều này rất quan trọng cho việc kiểm tra và debugging.
5. Tính Thời gian và Không gian (Temporal and Spatial Awareness):
- Mặc dù Scene Graph và Event Graph đã có thông tin thời gian/không gian, trong KG tổng quát, các mối quan hệ cũng có thể có thuộc tính thời gian (khi mối quan hệ đó tồn tại) và địa điểm (nơi mối quan hệ đó diễn ra). Điều này đặc biệt quan trọng cho các mối quan hệ sự kiện.
6. Xử lý Sự không chắc chắn (Uncertainty Handling):
- Duy trì confidence_score cho cả thực thể và mối quan hệ để biểu thị mức độ tin cậy của thông tin.



Cấu trúc cụ thể:
{
  "knowledgeGraphId": "<UUID duy nhất cho Knowledge Graph này (ví dụ: 'kg_unified_hatinh_flood_2023')>",
  "metadata": {
    "eventName": "<Tên sự kiện tổng quát (ví dụ: 'Lũ lụt Hà Tĩnh tháng 10/2023')>",
    "analysisDate": "<Ngày phân tích (ISO 8601, ví dụ: '2023-11-20T10:00:00Z')>",
    "description": "<Tóm tắt cụ thể về sự kiện (ví dụ: 'Knowledge Graph tổng hợp từ video tin tức về lũ lụt Hà Tĩnh, bao gồm thông tin hình ảnh và sự kiện.')>"
  },
  "globalEntities": [
    {
      "kgEntityId": "<UUID duy nhất cho thực thể toàn cầu này (ví dụ: 'kg_ent_ngoc_anh_singer')>",
      "type": "<Loại thực thể (ví dụ: 'person', 'object', 'organization', 'location')>",
      "subtype": "<Loại chi tiết hơn (ví dụ: 'singer', 'flood_event', 'main_road')>",
      "identity": {
        "name": "<Tên riêng được xác định rõ ràng (ví dụ: 'Ngọc Anh', 'Hà Tĩnh', 'Đường sắt Bắc-Nam')>",
        "label": "<Nhãn chung nhất (ví dụ: 'người', 'tỉnh', 'cơ sở hạ tầng')>",
        "brand": "<Thương hiệu nếu có>",
        "alias": ["<tên gọi khác 1 (ví dụ: 'ca sĩ Ngọc Anh')>", "<tên gọi khác 2>"]
      },
      "attributesUnified": { // Các thuộc tính được hợp nhất từ các nguồn khác nhau
        "physicalDescription": {
          "colorPalette": ["<màu 1 (ví dụ: 'tím sen')>", "<màu 2 (ví dụ: 'vàng')>"],
          "shape": "<hình dạng (ví dụ: 'humanoid', 'rectangular')>",
          "material": "<chất liệu (ví dụ: 'vải', 'kim loại')>",
          "sizeEstimation": "<ước lượng kích thước (ví dụ: 'medium', 'large')>"
        },
        "demographics": { // Nếu là người
          "gender": "<giới tính (ví dụ: 'nữ')>",
          "ageEstimation": "<độ tuổi ước tính (ví dụ: '20-25 tuổi')>"
        },
        "functionalDetails": "<chức năng tổng quát (ví dụ: 'khuếch đại âm thanh', 'vận chuyển hành khách')>",
        "contextualRoles": [ // Các vai trò khác nhau trong các ngữ cảnh khác nhau
          {"role": "ca sĩ chính", "context": "buổi biểu diễn"},
          {"role": "phó tổng giám đốc", "context": "Tổng công ty Đường sắt Việt Nam"},
          {"role": "tác nhân gây ra", "context": "sự kiện lũ lụt"}
        ],
        "status": "<trạng thái hiện tại hoặc chung (ví dụ: 'đang hoạt động', 'bị gián đoạn', 'đã khắc phục')>"
      },
      "sourceReferences": [ // Liên kết ngược về các ID gốc
        {"type": "scene_graph_entity", "id": "<UUID từ Scene Graph (ví dụ: 'ent_person_abc1')>", "frameNumber": "<số khung hình (ví dụ: 100)>"},
        {"type": "event_graph_actor", "id": "<UUID từ Event Graph actor (ví dụ: 'act_001_flood_water')>"},
        {"type": "event_graph_affected_entity", "id": "<UUID từ Event Graph affected entity (ví dụ: 'aff_008_nort_south_railway')>"}
      ],
      "provenance": {
        "source": ["<nguồn 1 (ví dụ: 'visual_recognition')>", "<nguồn 2 (ví dụ: 'asr')>", "<nguồn 3 (ví dụ: 'event_extraction')>"],
        "overallConfidence": "<Điểm tin cậy tổng thể của thực thể (ví dụ: 0.95)>"
      }
    }
  ],
  "globalRelations": [
    {
      "kgRelationId": "<UUID duy nhất cho mối quan hệ toàn cầu (ví dụ: 'kg_rel_wears_ngocanh_aodai')>",
      "subjectKgEntityId": "<ID của chủ thể trong globalEntities (ví dụ: 'kg_ent_ngoc_anh_singer')>",
      "predicateType": "<Loại quan hệ (ví dụ: 'wears', 'holds', 'causes', 'located_in', 'is_role_of', 'affected_by')>",
      "objectKgEntityId": "<ID của đối tượng trong globalEntities (ví dụ: 'kg_ent_ao_dai_tim_sen')>",
      "relationAttributes": {
        "timeContext": {"start": "<ISO 8601 (ví dụ: '2023-10-30T10:00:00Z')>", "end": "<ISO 8601>"},
        "locationContext": "<Địa điểm cụ thể của mối quan hệ (ví dụ: 'sân khấu')>",
        "causalStrength": "<Mức độ mạnh yếu của quan hệ nhân quả (ví dụ: 'high', 'medium')>"
      },
      "provenance": {
        "source": ["<nguồn 1 (ví dụ: 'scene_graph_relation')>", "<nguồn 2 (ví dụ: 'event_graph_relation')>", "<nguồn 3 (ví dụ: 'inferred')>"],
        "confidenceScore": "<Điểm tin cậy của mối quan hệ (ví dụ: 0.9)>"
      }
    }
  ],
  "ocr": { // Các thông tin OCR tổng hợp
    "related": ["<OCR liên quan 1 (ví dụ: 'HTV9 HD')>", "<OCR liên quan 2 (ví dụ: 'CHƯƠNG TRÌNH 60 GIÂY')>"],
    "notRelated": ["<OCR không liên quan 1 (ví dụ: 'quảng cáo')>"]
  },
  "eventTags": ["<tag 1 (ví dụ: 'thiên tai')>", "<tag 2 (ví dụ: 'lũ lụt')>", "<tag 3 (ví dụ: 'Hà Tĩnh')>"]
}
"""








role_prompt = """
Bạn là một chuyên gia trong việc phân tích hình ảnh, và thông tin chi tiết từ sự kiện. Nhiệm vụ của bạn, một cách tổng quan:
1. Từ các hình ảnh keyframe, mỗi hình ảnh, bạn phải tạo ra một caption thật chi tiết và đầy đủ. Một caption đầy đủ là một caption những thông tin liên quan tới bối cảnh (bối cảnh như thế nào, thiên nhiên, cảnh vật, hay hình nền, ...), thông tin liên quan tới những thực thể (là những thực thể có thể detect được, như một số con người, vật thể), hoặc cụm thực thể (là những thực thể có na ná các tính chất nhưng với số lượng nhiều). Kèm theo đó là các mối quan hệ về hành động, thông tin không gian(2d/3d) và các đặc tính cụ thể chi tiết cho từng thực thể/cụm thực thể.
2. Từ các caption mà bạn hoàn thành ở bước 1, hãy tạo ra một visual scene graph cụ thể cho từng keyframe. 
3. Sau khi có tất cả các visual caption và visual scene graph, từ đoạn ASR, bạn có thể tạo ra một caption event hoàn chỉnh. Từ đó bạn sẽ tạo ra một event graph hoàn chỉnh. Trong đó bạn phải thực hiện những việc như entity linking, entity resolution, action mapping cho từng thực thể/cum thực thể tương ứng với nội dung sự kiện. 
"""